# 1. ML

In [15]:
import os
import pandas as pd

# Dossier contenant les métriques
metrics_dir = 'Metrics'

# Liste pour stocker les DataFrames
dataframes = []

# Parcourir les pays et les modèles dans le répertoire Metrics
for country in os.listdir(metrics_dir):
    country_path = os.path.join(metrics_dir, country)
    
    # Vérifie si c'est un répertoire
    if os.path.isdir(country_path):  
        for model in os.listdir(country_path):
            model_path = os.path.join(country_path, model, 'Best_Fold_Metrics_ML.csv')
            
            # Vérifie si le fichier existe
            if os.path.isfile(model_path):
                # Lire le fichier CSV et ajouter une colonne pour le pays et le modèle
                df = pd.read_csv(model_path)
                
                # Ajouter des colonnes pour le pays et le modèle
                df['Country'] = country
                df['Model'] = model
                
                # Ajouter le DataFrame à la liste
                dataframes.append(df)
            else:
                print(f"Fichier manquant : {model_path}")  # Avertir si le fichier n'existe pas

# Vérifier si des DataFrames ont été collectés avant de les combiner
if dataframes:
    # Combiner tous les DataFrames en un seul
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Réorganiser les colonnes, en vérifiant si les colonnes existent
    column_order = ['Country', 'Algorithms', 'Fold', 'AUC', 'ACC', 'Sensitivity', 'Specificity', 'F1 Score', 'Model']
    combined_df = combined_df[column_order] if all(col in combined_df.columns for col in column_order) else combined_df

    # Sauvegarder le DataFrame combiné
    combined_file_path = os.path.join(metrics_dir, 'Combined_Best_Fold_Metrics_ML.csv')
    combined_df.to_csv(combined_file_path, index=False)

    # Afficher le DataFrame combiné
    print("Tableau des Métriques Combinées :")
    print(combined_df)  # Utilisez print pour afficher le DataFrame
else:
    print("Aucun DataFrame n'a été trouvé.")

# Si vous êtes dans un Jupyter Notebook, utilisez display pour un meilleur rendu
# display(combined_df)  # Décommenter si nécessaire


Fichier manquant : Metrics\IN\CNN\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\IN\CNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\IN\ML\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\IN\RNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\MG\CNN\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\MG\CNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\MG\ML\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\MG\RNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\PH\CNN\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\PH\CNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\PH\ML\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\PH\RNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\SA\CNN\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\SA\CNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\SA\ML\Best_Fold_Metrics_ML.csv
Fichier manquant : Metrics\SA\RNN_LSTM\Best_Fold_Metrics_ML.csv
Fichier manquant : M

# 2. CNN

In [16]:
import os
import pandas as pd
import glob

def combine_best_fold_metrics(base_dir, country, model_name):
    model_dir = os.path.join(base_dir, country, model_name)
    
    # Rechercher tous les fichiers CSV
    all_files_pattern = os.path.join(model_dir, f'All_Folds_Metrics_{model_name}_{country}.csv')
    all_files_pattern_2 = os.path.join(model_dir, 'All_Metrics*.csv')

    all_files = glob.glob(all_files_pattern) + glob.glob(all_files_pattern_2)

    print(f"Fichiers trouvés pour {country} - {model_name}: {all_files}")

    metrics_list = []

    for file in all_files:
        df = pd.read_csv(file)
        df['Country'] = country
        df['Algorithms'] = model_name
        
        # Vérifie les colonnes disponibles
        print(f"Colonnes dans le fichier {file}: {df.columns.tolist()}")
        
        # Vérification des données
        print("Données dans le fichier :")
        print(df.head())  # Afficher les premières lignes pour le diagnostic

        # Ajouter le numéro de fold à partir de la colonne 'Fold' si elle existe
        if 'Fold' in df.columns and 'AUC-ROC' in df.columns:
            best_fold_index = df['AUC-ROC'].idxmax()
            best_fold_metrics = df.iloc[best_fold_index]
            
            # Imprimer les métriques du meilleur fold pour vérification
            print(f"Métriques du meilleur fold pour {country}:")
            print(best_fold_metrics)

            metrics_list.append({
                'Country': country,
                'Algorithms': model_name,
                'Best_Fold': best_fold_metrics['Fold'],
                'AUC': best_fold_metrics['AUC-ROC'],
                'Accuracy': best_fold_metrics.get('Accuracy', 0),  # Utiliser le bon nom de colonne
                'Recall': best_fold_metrics.get('Recall', 0),  # Utiliser le bon nom de colonne
                'Specificity': best_fold_metrics.get('Specificity', 0),  # Utiliser le bon nom de colonne
                'F1 Score': best_fold_metrics.get('F1 Score', 0),  # Utiliser le bon nom de colonne
            })

    if not metrics_list:
        print(f"Aucun fichier à concaténer pour {country} - {model_name}")
        return pd.DataFrame()  # Retourne un DataFrame vide

    combined_metrics = pd.DataFrame(metrics_list)
    return combined_metrics

base_metrics_dir = 'Metrics'
models = ['CNN']  # Garder uniquement le modèle CNN
all_combined_metrics = []
countries = ['VN', 'PH', 'MG', 'IN', 'SA', 'TZ', 'UG']

for country in countries:
    combined_df = combine_best_fold_metrics(base_metrics_dir, country, models[0])
    if not combined_df.empty:
        all_combined_metrics.append(combined_df)

if all_combined_metrics:
    final_combined_df = pd.concat(all_combined_metrics, ignore_index=True)

    # Afficher les colonnes disponibles dans le DataFrame
    print("Colonnes disponibles dans le DataFrame combiné :")
    print(final_combined_df.columns)

    # Afficher uniquement les colonnes existantes
    columns_to_display = ['Country', 'Algorithms', 'Best_Fold', 'AUC', 'Accuracy', 'Recall', 'Specificity', 'F1 Score']
    available_columns = [col for col in columns_to_display if col in final_combined_df.columns]
    
    print("Métriques combinées par pays et modèle pour le meilleur fold :")
    print(final_combined_df[available_columns])
else:
    print("Aucune métrique combinée trouvée.")

# Afficher le DataFrame final avec les colonnes disponibles
final_combined_df[available_columns]


Fichiers trouvés pour VN - CNN: ['Metrics\\VN\\CNN\\All_Folds_Metrics_CNN_VN.csv']
Colonnes dans le fichier Metrics\VN\CNN\All_Folds_Metrics_CNN_VN.csv: ['Fold', 'Model', 'Accuracy', 'Recall', 'F1 Score', 'Specificity', 'AUC-ROC', 'Country', 'Algorithms']
Données dans le fichier :
   Fold Model  Accuracy    Recall  F1 Score  Specificity   AUC-ROC Country  \
0     1   CNN  0.615385  0.000000  0.000000          1.0  0.550000      VN   
1     2   CNN  0.615385  0.000000  0.000000          1.0  0.493750      VN   
2     3   CNN  0.538462  0.727273  0.571429          0.4  0.472727      VN   
3     4   CNN  0.600000  0.000000  0.000000          1.0  0.600000      VN   
4     5   CNN  0.600000  0.000000  0.000000          1.0  0.540000      VN   

  Algorithms  
0        CNN  
1        CNN  
2        CNN  
3        CNN  
4        CNN  
Métriques du meilleur fold pour VN:
Fold             4
Model          CNN
Accuracy       0.6
Recall         0.0
F1 Score       0.0
Specificity    1.0
AUC-ROC  

,Country,Algorithms,Best_Fold,AUC,Accuracy,Recall,Specificity,F1 Score
0,VN,CNN,4,0.600000,0.600000,0.000000,1.000000,0.000000
1,PH,CNN,5,0.641975,0.933333,0.333333,1.000000,0.500000
2,MG,CNN,5,0.730769,0.560000,0.083333,1.000000,0.153846
3,IN,CNN,5,0.875000,0.111111,1.000000,0.000000,0.200000
4,SA,CNN,1,0.764706,0.409091,1.000000,0.235294,0.434783
5,TZ,CNN,1,0.833333,0.785714,1.000000,0.750000,0.571429
6,UG,CNN,1,0.686154,0.526316,0.923077,0.320000,0.571429


# Prédiction par pays et par modèle

In [ ]:
yyy

In [ ]:
/

In [17]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Afficher le tableau final
print(pivoted_df)
pivoted_df


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture 

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
*

In [4]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Ajouter une ligne vide entre les pays
separator_rows = []
for country in countries:
    country_rows = pivoted_df[pivoted_df['Country'] == country]
    separator_rows.append(country_rows)
    separator_rows.append(pd.DataFrame(columns=pivoted_df.columns))  # Ajouter une ligne vide

# Combiner toutes les lignes
final_separated_df = pd.concat(separator_rows, ignore_index=True)

# Afficher le tableau final avec séparation
print(final_separated_df)
final_separated_df


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture 

TypeError: agg function failed [how->mean,dtype->object]

In [5]:
import pandas as pd
import os
import glob
from IPython.display import display

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Afficher des tableaux pandas pour chaque pays
for country in countries:
    country_df = pivoted_df[pivoted_df['Country'] == country]
    print(f"\nTableau pour le pays : {country}")
    display(country_df)


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture de Me

Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
26,VN,ML,0.336538,0.363636,0.086957,0.076923,0.55
24,VN,CNN,0.773077,0.360000,0.636364,0.538462,0.90
25,VN,CNN_LSTM,0.603846,0.615385,0.000000,0.000000,1.00
27,VN,RNN_LSTM,0.561538,0.600000,0.000000,0.000000,1.00



Tableau pour le pays : PH


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
10,PH,ML,0.585714,0.666667,0.133333,0.25,0.714286
8,PH,CNN,0.457143,0.800000,0.111111,0.25,0.628571
9,PH,CNN_LSTM,0.421429,0.900000,0.000000,0.00,1.000000
11,PH,RNN_LSTM,0.771429,0.900000,0.000000,0.00,1.000000



Tableau pour le pays : SA


...

Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
6,MG,ML,0.635294,0.59375,0.551724,0.533333,0.647059
4,MG,CNN,0.658824,0.48000,0.000000,0.000000,1.000000
5,MG,CNN_LSTM,0.419608,0.72000,0.540541,0.666667,0.294118
7,MG,RNN_LSTM,0.596078,0.52000,0.551724,0.533333,0.647059



Tableau pour le pays : TZ


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
18,TZ,ML,0.866667,0.277778,0.133333,0.333333,0.266667
16,TZ,CNN,0.422222,0.846154,0.235294,0.666667,0.200000
17,TZ,CNN_LSTM,0.244444,0.857143,0.000000,0.000000,0.933333
19,TZ,RNN_LSTM,0.422222,0.846154,0.000000,0.000000,1.000000



Tableau pour le pays : UG


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
22,UG,ML,0.667339,0.659574,0.60000,0.7500,0.612903
20,UG,CNN,0.375000,0.324324,0.27027,0.3125,0.483871
21,UG,CNN_LSTM,0.588710,0.648649,0.00000,0.0000,1.000000
23,UG,RNN_LSTM,0.491935,0.648649,0.00000,0.0000,1.000000


In [6]:
import pandas as pd
import os
import glob
from IPython.display import display

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Sélectionner le meilleur modèle par pays en fonction d'une métrique, ici 'Accuracy'
best_models = pivoted_df.loc[pivoted_df.groupby('Country')['Accuracy'].idxmax()]

# Afficher des tableaux pandas pour chaque pays
for country in countries:
    country_df = best_models[best_models['Country'] == country]
    print(f"\nTableau pour le pays : {country}")
    display(country_df)


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture de Me

Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
25,VN,CNN_LSTM,0.603846,0.615385,0.0,0.0,1.0



Tableau pour le pays : PH


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
9,PH,CNN_LSTM,0.421429,0.9,0.0,0.0,1.0



Tableau pour le pays : SA


...

Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
5,MG,CNN_LSTM,0.419608,0.72,0.540541,0.666667,0.294118



Tableau pour le pays : TZ


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
17,TZ,CNN_LSTM,0.244444,0.857143,0.0,0.0,0.933333



Tableau pour le pays : UG


Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
22,UG,ML,0.667339,0.659574,0.6,0.75,0.612903


In [5]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Sélectionner le meilleur modèle par pays en fonction d'une métrique, ici 'Accuracy'
best_models = pivoted_df.loc[pivoted_df.groupby('Country')['Accuracy'].idxmax()]

# Combiner tous les meilleurs modèles en un seul tableau
final_best_models_table = best_models.reset_index(drop=True)

# Afficher le tableau combiné
print("\nTableau combiné des meilleurs modèles par pays :")
display(final_best_models_table)


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture 

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
***************************************

In [7]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Sélectionner le meilleur modèle par pays en fonction de l'AUC-ROC
best_models = pivoted_df.loc[pivoted_df.groupby('Country')['AUC-ROC'].idxmax()]

# Combiner tous les meilleurs modèles en un seul tableau
final_best_models_table = best_models.reset_index(drop=True)

# Afficher le tableau combiné
print("\nTableau combiné des meilleurs modèles par pays :")
final_best_models_table


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture de Me

Metric,Country,Model,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
0,IN,ML,0.523810,0.750000,0.000000,0.000000,0.857143
1,MG,CNN,0.658824,0.480000,0.000000,0.000000,1.000000
2,PH,RNN_LSTM,0.771429,0.900000,0.000000,0.000000,1.000000
3,SA,RNN_LSTM,0.575758,0.809524,0.000000,0.000000,1.000000
4,TZ,ML,0.866667,0.277778,0.133333,0.333333,0.266667
5,UG,ML,0.667339,0.659574,0.600000,0.750000,0.612903
6,VN,CNN,0.773077,0.360000,0.636364,0.538462,0.900000


In [ ]:
-----------------------------------------------------------------------------------------------

In [21]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            final_metrics.append(df)

        # Vérifier et lire des fichiers spécifiques pour CNN_LSTM et RNN_LSTM
        if model_type == "CNN_LSTM":
            alt_pattern = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(alt_pattern):
                print(f"Lecture de {alt_pattern} pour {country} - CNN_LSTM")
                alt_df = pd.read_csv(alt_pattern)
                alt_df['Country'] = country
                alt_df['Model'] = 'CNN_LSTM'
                final_metrics.append(alt_df)

        if model_type == "RNN_LSTM":
            additional_patterns = [
                os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
                os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv")
            ]
            for alt_pattern in additional_patterns:
                if os.path.exists(alt_pattern):
                    print(f"Lecture de {alt_pattern} pour {country} - RNN_LSTM")
                    alt_df = pd.read_csv(alt_pattern)
                    alt_df['Country'] = country
                    alt_df['Model'] = 'RNN_LSTM'
                    final_metrics.append(alt_df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays et modèles en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model'],
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Sélectionner le meilleur modèle par pays en fonction de l'AUC-ROC
best_models = pivoted_df.loc[pivoted_df.groupby('Country')['AUC-ROC'].idxmax()]

# Combiner tous les meilleurs modèles en un seul tableau
final_best_models_table = best_models.reset_index(drop=True)

# Afficher le tableau combiné
print("\nTableau combiné des meilleurs modèles par pays :")
print(final_best_models_table)


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture 

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
************************

# FIN

In [9]:
import pandas as pd
import os
import glob

# Définir le chemin de base et les pays
base_path = 'Metrics'
countries = ['VN', 'PH', 'SA', 'IN', 'MG', 'TZ', 'UG']
model_types = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']  # Inclure 'ML' ici

# Liste pour stocker les métriques finales
final_metrics = []

# Dictionnaire des algorithmes associés aux modèles
algorithms = {
    'ML': 'Random Forest',  # Exemple d'algorithme pour ML
    'CNN': 'CNN',
    'CNN_LSTM': 'CNN avec LSTM',
    'RNN_LSTM': 'RNN avec LSTM'
}

# Parcourir chaque pays
for country in countries:
    # Lire le fichier ML spécifique au pays
    ml_path = os.path.join(base_path, country, 'ML', 'Predictions_Metrics_ML.csv')
    if os.path.exists(ml_path):
        ml_df = pd.read_csv(ml_path)
        ml_df['Country'] = country
        ml_df['Model'] = 'ML'
        ml_df['Algorithm'] = algorithms['ML']  # Ajouter le nom de l'algorithme
        final_metrics.append(ml_df)

    # Parcourir les modèles pour chaque pays
    for model_type in model_types[1:]:  # Commencer par CNN
        # Motif pour les fichiers de métriques
        pattern = os.path.join(base_path, country, model_type, f"Predictions_Metrics_*_{country}.csv")
        
        # Ajout de motifs spécifiques pour CNN_LSTM
        if model_type == "CNN_LSTM":
            pattern_alt = os.path.join(base_path, country, "CNN_LSTM", f"Predictions_Test_Metrics_CNN_LSTM_{country}.csv")
            if os.path.exists(pattern_alt):
                pattern = pattern_alt

        # Ajouter des fichiers supplémentaires manquants
        additional_patterns = [
            os.path.join(base_path, country, f"Predictions_Test_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_RNN_LSTM_{country}.csv"),
            os.path.join(base_path, country, f"Predictions_Metrics_CNN_LSTM_{country}.csv")
        ]
        
        # Vérifier et ajouter des fichiers supplémentaires
        for alt_pattern in additional_patterns:
            if os.path.exists(alt_pattern):
                pattern = alt_pattern

        # Lire tous les fichiers correspondant au motif
        for file_path in glob.glob(pattern):
            print(f"Lecture de {file_path} pour {country} - {model_type}")
            df = pd.read_csv(file_path)
            df['Country'] = country
            df['Model'] = model_type
            df['Algorithm'] = algorithms[model_type]  # Ajouter le nom de l'algorithme

            # Convertir 'Value' en numérique, en forçant les erreurs à NaN
            df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

            # Supprimer les lignes avec des valeurs NaN dans la colonne 'Value'
            df.dropna(subset=['Value'], inplace=True)

            final_metrics.append(df)

# Combiner les résultats en un seul DataFrame
final_metrics_df = pd.concat(final_metrics, ignore_index=True)

# Afficher les colonnes du DataFrame combiné
print("Colonnes du DataFrame combiné :", final_metrics_df.columns.tolist())

# Pivot pour obtenir les pays, modèles et algorithmes en lignes, et les métriques en colonnes
pivoted_df = final_metrics_df.pivot_table(
    index=['Country', 'Model', 'Algorithm'],  # Inclure 'Algorithm' ici
    columns='Metric',
    values='Value',
    aggfunc='mean'
).reset_index()

# Définir l'ordre souhaité pour les modèles
model_order = ['ML', 'CNN', 'CNN_LSTM', 'RNN_LSTM']

# Créer une colonne d'ordre pour le tri
pivoted_df['Model_Order'] = pivoted_df['Model'].apply(lambda x: model_order.index(x))

# Trier le DataFrame par pays et ordre des modèles
pivoted_df = pivoted_df.sort_values(by=['Country', 'Model_Order']).drop(columns='Model_Order')

# Sélectionner le meilleur modèle par pays en fonction de l'AUC-ROC
best_models = pivoted_df.loc[pivoted_df.groupby('Country')['AUC-ROC'].idxmax()]

# Combiner tous les meilleurs modèles en un seul tableau
final_best_models_table = best_models.reset_index(drop=True)

# Afficher le tableau combiné
print("\nTableau combiné des meilleurs modèles par pays :")
final_best_models_table


Lecture de Metrics\VN\CNN\Predictions_Metrics_CNN_VN.csv pour VN - CNN
Lecture de Metrics\VN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_VN.csv pour VN - CNN_LSTM
Lecture de Metrics\VN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_VN.csv pour VN - RNN_LSTM
Lecture de Metrics\PH\CNN\Predictions_Metrics_CNN_PH.csv pour PH - CNN
Lecture de Metrics\PH\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_PH.csv pour PH - CNN_LSTM
Lecture de Metrics\PH\RNN_LSTM\Predictions_Metrics_RNN_LSTM_PH.csv pour PH - RNN_LSTM
Lecture de Metrics\SA\CNN\Predictions_Metrics_CNN_SA.csv pour SA - CNN
Lecture de Metrics\SA\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_SA.csv pour SA - CNN_LSTM
Lecture de Metrics\SA\RNN_LSTM\Predictions_Metrics_RNN_LSTM_SA.csv pour SA - RNN_LSTM
Lecture de Metrics\IN\CNN\Predictions_Metrics_CNN_IN.csv pour IN - CNN
Lecture de Metrics\IN\CNN_LSTM\Predictions_Test_Metrics_CNN_LSTM_IN.csv pour IN - CNN_LSTM
Lecture de Metrics\IN\RNN_LSTM\Predictions_Metrics_RNN_LSTM_IN.csv pour IN - RNN_LSTM
Lecture 

Metric,Country,Model,Algorithm,AUC,AUC-ROC,Accuracy,F1 Score,Recall,Specificity
0,IN,CNN,CNN,NaN,0.777778,0.500000,0.333333,1.000000,0.428571
1,MG,CNN,CNN,NaN,0.654902,0.593750,0.518519,0.466667,0.705882
2,PH,CNN,CNN,NaN,0.707143,0.743590,0.285714,0.500000,0.771429
3,SA,RNN_LSTM,RNN avec LSTM,NaN,0.507576,0.642857,0.166667,0.166667,0.772727
4,TZ,CNN,CNN,NaN,0.555556,0.611111,0.363636,0.666667,0.600000
5,UG,CNN_LSTM,CNN avec LSTM,NaN,0.532258,0.553191,0.400000,0.437500,0.612903
6,VN,CNN,CNN,NaN,0.723077,0.696970,0.444444,0.307692,0.950000
